## Let's build a class that defines a *feed-forwad artificial neural network*. 

Such "network" is basically a (in principle) very complicated function, with (in principle) a lot of adjastable parameters that is used to approximate (or interpolate) functions.  

A neural network is made up form neurons that are connected with each other. A feed-forwad artificial neural network takes inputs, transforms them by passing them through its "hidden" layer, and produces outputs.

A feed-forwad artificial neural network looks loke the following figure:

<img src="misc/FFANN.png" style="height:250px">

where the triangles represent the action of the neuron (some simple function we define), the coloured circles represent the "weights", and the empty ones represent the "biases". The weights and biases are adjustable parameters that we tune, in order to get the outputs we need from given inputs.  We should note that neurons do not connect with all the other ones, but the entire network is separated in "layers", with each layer taking as input the output of the previous one.

Every neuron takes a number of inputs coming from the previous layer. The neuron, then does the following:

<img src="misc/Neuron.png" style="height:450px">

That is, it takes the outputs from the previous layer ($l$), and the weights that represent the strength of the connection between the node $i$ of the $l$ layer and node $j$ of the $l+1$ layer (all these inputs and outputs are called "signals"). Then it adds all the different contributions and returns the output of which is:
$$
x^{(l)}_{j} = \theta\left( \displaystyle \sum_{i=0}^{n^{(l)}-1} w^{(l)}_{j i}  x^{(l)}_{i} + b^{(l+1)}_{j}\right) \; , 
$$

with $\theta$ the "activation function", which is some simple function we choose.


In [1]:
import numpy as np

In [2]:
class FFANN:
    def __init__(self, _inputs,_outputs,_hidden_nodes, 
                 _hidden_func=lambda x:x, _hidden_func_deriv=lambda x:1,
                 _output_func=lambda x:x, _output_func_deriv=lambda x:1):
        '''
        Constructor for the class.

        _inputs: number of input nodes
        _outputs: number of ouput nodes
        
        _hidden_nodes: list of number of layers in each hidden node
        _hidden_func,_output_func: activation functions for hidden layers and output, respectively
        _hidden_func_deriv,_output_func_deriv: derivatives of activation functions for hidden layers and output, respectively
        '''
        
        self.inputs=_inputs
        self.outputs=_outputs
        self.layers=len(_hidden_nodes)
        self.total_layers=2+self.layers #total layers = No. layers + input layer + output layer
        
        self.hidden_activation=_hidden_func
        self.hidden_activation_deriv=_hidden_func_deriv
        self.output_activation=_output_func
        self.output_activation_deriv=_output_func_deriv

        #an array to hold number of nodes at each layer
        self.nodes=[_inputs]
        
        for _n in _hidden_nodes:
            self.nodes.append(_n)
            
        self.nodes.append(_outputs)
        
            
        
        #declare the signals
        self.signals=[ [0 for j in range(self.nodes[l])]  for l in range(self.total_layers)]
        #in case you get confused, these are the indices
        self.signal_indices=[ ['x^({0})_{1}'.format(l,j) for j in range(self.nodes[l])]  for l in range(self.total_layers)]
        

        
        #declare the weights
        self.weights=[ [[0 for i in range(self.nodes[l])] for j in range(self.nodes[l+1])]  for l in range(self.layers+1)]
        #in case you get confused, these are the indices
        self.weight_indices=[ [['w^({0})_{1}{2}'.format(l,j,i) for i in range(self.nodes[l])] for j in range(self.nodes[l+1])]  for l in range(self.layers+1)]
        
        
        
        #declare the biases
        #By definition the biases of l=0 are zero, so we define biases[l][j] to refer to the l+1 layer.
        #this will help as we will be able to put the biases in the same loop as the weights (see init_params for example).
        self.biases=[ [0 for j in range(self.nodes[l+1])]  for l in range(self.layers+1)]        
        #in case you get confused, these are the indices
        self.bias_indices=[ ['b^({0})_{1}'.format(l+1,j) for j in range(self.nodes[l+1])]  for l in range(self.layers+1)]
        
    
        self.init_params()#initialize the weights and biases
                
    #========================initializations========================#
    def init_params(self):
        '''
        Function that initializes the weights and biasesby some random numbers.
        '''
        for l in range(self.layers+1):
            for j in range(self.nodes[l+1]):
                #You see, we defined biases in this way in order to manipulate them with the weights at the same loop. 
                self.biases[l][j] = (np.random.choice([-1,1])*np.random.random() )
                for i in range(self.nodes[l]):
                    self.weights[l][j][i] = (np.random.choice([-1,1])*np.random.random() )
    
    
    def write_params(self):
        '''
            Returns biases and weights as a vector.
            Can be used to pass the parameters to an optimization algorithm that takes vectors. 
        '''
        X=[]
        for l in range(self.layers+1):
            for j in range(self.nodes[l+1]):
                X.append(self.biases[l][j])
                for i in range(self.nodes[l]):
                    X.append(self.weights[l][j][i])
        return X
    
    def read_params(self,X):
        '''
            Read and update biases and weights from a vector X
            Can be used to read the result from an optimization algorithm that returns vectors. 
        '''
        _s=0
        for l in range(self.layers+1):
            for j in range(self.nodes[l+1]):
                self.update_bias(l,j,X[_s])
                _s+=1
                for i in range(self.nodes[l]):
                    self.update_weight(l,j,i,X[_s])
                    _s+=1
    
    def input_signal(self, x):
        '''Read the input (x).'''
        for j,_ in enumerate(x):
            self.signals[0][j] = _
    
    def calc_signal(self, l,j):
        '''calculate the output of the j node of layer l.'''
        if l == 0:
            print('for l=0 run self.input_signal')
            return 0
        
        sum_wx = np.sum( [ self.weights[l-1][j][i] * xi for i,xi in enumerate(self.signals[l-1]) ] ) 
        
        #Notice that self.biases[l-1][j] correspond to the bias of node j and layer l.
        if l == self.total_layers-1:
            self.signals[l][j] =  self.output_activation( sum_wx + self.biases[l-1][j]  )
            return 0
        
        if l < self.total_layers-1:
            self.signals[l][j] = self.hidden_activation( sum_wx + self.biases[l-1][j]  )
            return 0    
    
    #========================Feed-Forward========================#
    
    def feed_forward(self):
        '''
        Calculates the output of the network.
        Also sets the signals (for later use)
        '''
        for l in range(1,self.total_layers):
            for j in range(self.nodes[l]):
                self.calc_signal(l,j)
            
            
            
            
    #========================update========================#

    def update_weight(self,l,j,i,value):
        '''
        Change the value of w^{l}_{ji} to value
        '''
        self.weights[l][j][i]=value

    def update_bias(self,l,j,value):
        '''
        Change the value of b^{l}_{j} to value
        '''
        self.biases[l][j]=value
    #======================================================#


In [3]:
brain = FFANN(2,1,[3])

In [4]:
#this is how you run it!
brain.input_signal([2,1])
brain.feed_forward()
brain.signals

[[2, 1],
 [0.5333569489144613, 0.34008546591749245, 2.412298499002562],
 [0.060177639527672855]]

### Check that the read and write to vector do not mess-up the biases and matrices

In [5]:
brain = FFANN(2,1,[1,3])

In [6]:
brain.weights,brain.biases

([[[-0.20366425972085334, 0.818560173679066]],
  [[0.34308224697805345], [-0.26406907419433423], [0.6200973023335733]],
  [[0.2725832433397404, -0.7802710900794901, -0.8475099534208587]]],
 [[0.8983739866766525],
  [-0.5023652620355369, 0.3863775479380248, -0.4436935960669719],
  [-0.11896976767245182]])

In [7]:
X=brain.write_params()

In [8]:
brain.read_params(X)

In [9]:
brain.weights,brain.biases

([[[-0.20366425972085334, 0.818560173679066]],
  [[0.34308224697805345], [-0.26406907419433423], [0.6200973023335733]],
  [[0.2725832433397404, -0.7802710900794901, -0.8475099534208587]]],
 [[0.8983739866766525],
  [-0.5023652620355369, 0.3863775479380248, -0.4436935960669719],
  [-0.11896976767245182]])